# CLIF to C2D2 Dataset

Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pyCLIF
from datetime import timedelta
import pyarrow
import waterfall
import warnings
warnings.filterwarnings('ignore')

output_folder = '../output/final'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

✅ Loaded configuration from config.json


In [2]:
clif_c2d2_mapping = pd.read_excel('../mapping/ccm-53-e1045-s002.xlsx')

## 🏥 ICU Cohort Generation

The ICU cohort is derived by identifying hospitalizations that meet the following criteria:

1. **Adult patients** (age ≥ 18 years at time of admission)
2. **Received ICU-level care** during hospitalization
3. **Completed at least 24 consecutive hours** in an ICU location

### Key Steps:
- Extract ICU admission and discharge timestamps from ICU transfer logs
- Sort ICU stays chronologically within each hospitalization
- Identify and retain the **first ICU stay** that meets the 24-hour requirement
- Generate additional variables:
  - `24hr_timestamp`: 24 hours from ICU admission
  - `prism_2hr_before_icu_admission`: 2 hours prior to ICU admission
  - `prism_4hr_after_icu_admission`: 4 hours post ICU admission
  - `all_icu_stays`: list of all ICU in/out pairs
  - `total_icu_stay_minutes`: cumulative ICU duration (excluding negative intervals)

This curated ICU cohort is then merged with **hospitalization** and **patient-level** tables to enrich demographic and clinical context.


### Load Data

In [3]:
patient = pyCLIF.load_data('clif_patient')
hospitalization = pyCLIF.load_data('clif_hospitalization')
adt = pyCLIF.load_data('clif_adt')

# ensure id variable is of dtype character
hospitalization['hospitalization_id']= hospitalization['hospitalization_id'].astype(str)
patient['patient_id']= patient['patient_id'].astype(str)
adt['hospitalization_id']= adt['hospitalization_id'].astype(str)

Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet_2\clif_patient.parquet


Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet_2\clif_hospitalization.parquet


Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet_2\clif_adt.parquet


### 🔁 Duplicate Check & 🕒 TimeZone Conversion (QA)

- **Duplicate Handling**:  
  If duplicates are present, only the **first row** is retained after sorting records by **chronological order (e.g., `in_dttm`)**.  
  Please inspect your **CLIF tables** for duplicate entries, particularly in key event logs such as ICU admissions.

- **TimeZone Conversion**:  
  Ensure all timestamps are consistently converted to the **desired standard timezone** (e.g., UTC or hospital-local time).  
  This is critical for temporal alignment across tables and for accurate window-based feature extraction.


In [4]:
# check for duplicates
# patient table should be unique by patient id
patient = pyCLIF.remove_duplicates(patient, ['patient_id'], 'patient')
# hospitalization table should be unique by hospitalization id
hospitalization = pyCLIF.remove_duplicates(hospitalization, ['hospitalization_id'], 'hospitalization')
# adt table should be unique by hospitalization id and in dttm
adt = pyCLIF.remove_duplicates(adt, ['hospitalization_id', 'hospital_id', 'in_dttm'], 'adt')

Processing DataFrame: patient
No duplicates found based on columns: ['patient_id'].
Processing DataFrame: hospitalization
No duplicates found based on columns: ['hospitalization_id'].


Processing DataFrame: adt
No duplicates found based on columns: ['hospitalization_id', 'hospital_id', 'in_dttm'].


In [5]:
print(f"Total Number of unique encounters in the hospitalization table: {pyCLIF.count_unique_encounters(hospitalization, 'hospitalization_id')}")

Total Number of unique encounters in the hospitalization table: 227693


In [6]:
# Standardize all _dttm variables to the same format
patient = pyCLIF.convert_datetime_columns_to_site_tz(patient,  pyCLIF.helper['timezone'])
hospitalization = pyCLIF.convert_datetime_columns_to_site_tz(hospitalization, pyCLIF.helper['timezone'])
adt = pyCLIF.convert_datetime_columns_to_site_tz(adt,  pyCLIF.helper['timezone'])

### 🧬 Cohort Identification

#### ✅ Inclusion Criteria
- **Adult hospital admissions**
- **Received care in the ICU** during hospitalization
- **Completed at least 24 consecutive hours** of ICU stay


In [7]:
icu_adt = adt[adt['location_category'].str.lower()=='icu']

# Calculate minutes between timestamps
icu_adt['icu_duration_minutes'] = (icu_adt['out_dttm'] - icu_adt['in_dttm']).dt.total_seconds() / 60

# icu adt with at least 24 hour stay
icu_adt = icu_adt[icu_adt['icu_duration_minutes']>=1440].copy()

In [8]:
# Get all ICU stays as a list of (in_dttm, out_dttm) tuples per hospitalization_id
icu_stays_grouped = (
    icu_adt
    .sort_values('in_dttm')
    .groupby('hospitalization_id')
    .apply(lambda df: list(zip(df['in_dttm'], df['out_dttm'])))
    .reset_index()
    .rename(columns={0: 'all_icu_stays'})
)

# Get the first ICU admission per hospitalization_id
first_24hr_icu_per_hosp = (
    icu_adt
    .sort_values('in_dttm')
    .groupby('hospitalization_id')
    .first()
    .reset_index()
)

# Add relevant time-based columns
first_24hr_icu_per_hosp['24hr_timestamp'] = first_24hr_icu_per_hosp['in_dttm'] + timedelta(hours=24)
first_24hr_icu_per_hosp['prism_2hr_before_icu_admission'] = first_24hr_icu_per_hosp['in_dttm'] - timedelta(hours=2)
first_24hr_icu_per_hosp['prism_4hr_after_icu_admission'] = first_24hr_icu_per_hosp['in_dttm'] + timedelta(hours=4)

# Merge the all_icu_stays column into the same table
first_24hr_icu_per_hosp = pd.merge(
    first_24hr_icu_per_hosp,
    icu_stays_grouped,
    how='left',
    on='hospitalization_id'
)


In [9]:
# icu_los
def compute_total_icu_minutes(stays):
    if isinstance(stays, list) and stays:
        return sum(
            (out - inp).total_seconds() / 60
            for inp, out in stays
            if (out - inp).total_seconds() > 0
        )
    return 0


first_24hr_icu_per_hosp['total_icu_stay_minutes'] = first_24hr_icu_per_hosp['all_icu_stays'].apply(compute_total_icu_minutes)

In [10]:
# First inner join with hospitalization on 'hospitalization_id'
icu_with_hosp = pd.merge(
    first_24hr_icu_per_hosp,
    hospitalization,
    how='inner',
    on='hospitalization_id'
)

# Then inner join with patient on 'patient_id'
icu_with_hosp_and_patient = pd.merge(
    icu_with_hosp,
    patient,
    how='inner',
    on='patient_id'
)

icu_with_hosp_and_patient['hosp_los'] = (
    icu_with_hosp_and_patient['discharge_dttm'] - icu_with_hosp_and_patient['admission_dttm']
).dt.total_seconds() / 60  # Optional: divide by 60 to convert to minutes


In [11]:
cohort = icu_with_hosp_and_patient[['hospitalization_id','in_dttm',
                                    'out_dttm', 'location_category', 'hospital_id',
                                    'icu_duration_minutes', '24hr_timestamp',
                                    'prism_2hr_before_icu_admission', 'prism_4hr_after_icu_admission', 'admission_dttm', 'discharge_dttm',
                                    'all_icu_stays','total_icu_stay_minutes','hosp_los','age_at_admission','admission_type_category', 
                                    'discharge_category','sex_category',  'race_category', 'ethnicity_category']]

### 🗺️ Map to C2D2 Standard Values

This step ensures alignment with the **C2D2 data harmonization framework**. All relevant fields in the dataset are mapped to their corresponding **C2D2 standard values**, including:

- **Admission and discharge types**
- **ICU location categories**
- **Demographics** (sex, race, ethnicity)
- **Hospital and unit identifiers**
- **Temporal markers** (e.g., admission timestamps, ICU transitions)

Mappings should follow the **latest C2D2 dictionary** and be version-controlled to maintain auditability and reproducibility.


In [12]:
cohort['sex_category'].value_counts()

sex_category
Male      22478
Female    20472
Name: count, dtype: int64

In [13]:
cohort['race_category'].value_counts()

race_category
White                                        17193
Black or African American                    16395
Other                                         7100
Asian                                         1382
Unknown                                        719
American Indian or Alaska Native               113
Native Hawaiian or Other Pacific Islander       48
Name: count, dtype: int64

In [14]:
ethnicity_mapping = {
    'Hispanic': 'Hispanic or Latino',
    'Non-Hispanic': 'Non-Hispanic or non-Latino',
    'Unknown': 'Unknown'
}

cohort['ethnicity_category'] = cohort['ethnicity_category'].map(ethnicity_mapping)


cohort['ethnicity_category'].value_counts()

ethnicity_category
Non-Hispanic or non-Latino    34384
Hispanic or Latino             8090
Unknown                         476
Name: count, dtype: int64

In [15]:
disposition_mapping = {
    'Home': 'Home',
    'Against Medical Advice (AMA)': 'Discharged to Other Institution',
    'Hospice': 'Discharged to Other Institution',
    'Skilled Nursing Facility (SNF)': 'Discharged to Other Institution',
    'Acute Inpatient Rehab Facility': 'Discharged to Other Institution',
    'Expired': 'Deceased',
    'Long Term Care Hospital (LTACH)': 'Discharged to Other Institution',
    'Acute Care Hospital': 'Discharged to Other Institution',
    'Group Home': 'Discharged to Other Institution',
    'Psychiatric Hospital': 'Discharged to Other Institution',
    'Jail': 'Discharged to Other Institution',
    'Missing': 'Discharged to Other Institution',
    'Other': 'Discharged to Other Institution',
    'Chemical Dependency': 'Discharged to Other Institution',
    'Assisted Living': 'Discharged to Other Institution',
    'Still Admitted': 'Discharged to Other Institution'
}
cohort['discharge_category'] = cohort['discharge_category'].map(disposition_mapping)
cohort['discharge_category'].value_counts()


discharge_category
Home                               25494
Discharged to Other Institution    14399
Deceased                            3057
Name: count, dtype: int64

In [16]:
cohort.columns

Index(['hospitalization_id', 'in_dttm', 'out_dttm', 'location_category',
       'hospital_id', 'icu_duration_minutes', '24hr_timestamp',
       'prism_2hr_before_icu_admission', 'prism_4hr_after_icu_admission',
       'admission_dttm', 'discharge_dttm', 'all_icu_stays',
       'total_icu_stay_minutes', 'hosp_los', 'age_at_admission',
       'admission_type_category', 'discharge_category', 'sex_category',
       'race_category', 'ethnicity_category'],
      dtype='object')

In [17]:
rename_columns = {
    'hospitalization_id': 'hospitalization_id',
    'in_dttm': '24hr_icu_in_dttm',
    'out_dttm': 'icu_out_dttm',
    'location_category': 'location_category',
    'hospital_id': 'hospital_id',
    'icu_duration_minutes': 'icu_duration_minutes',
    '24hr_timestamp': '24hr_icu_out_dttm',
    'prism_2hr_before_icu_admission': 'prism_2hr_before_icu_admission',
    'prism_4hr_after_icu_admission': 'prism_4hr_after_icu_admission',
    'all_icu_stays': 'all_icu_stays',
    'total_icu_stay_minutes': 'icu_los',
    'hosp_los': 'hosp_los',
    'age_at_admission': 'icu_admit_age',
    'admission_type_category': 'hosp_admit_source',
    'discharge_category': 'hosp_disch_disp',
    'sex_category': 'sex',
    'race_category': 'race',
    'ethnicity_category': 'ethnicity'
}

cohort = cohort.rename(columns=rename_columns)

# los in days as per c2d2
cohort['icu_los'] = cohort['icu_los'] / (60 * 24)
cohort['hosp_los'] = cohort['hosp_los'] / (60 * 24)

In [18]:
cohort.to_parquet('../output/final/c2d2_cohort.parquet',index=False)